In [ ]:
import pandas as pd
import numpy as np
import  matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats


from datetime import datetime
from numpy.polynomial.polynomial import Polynomial
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler

# Pertanyaan Bisnis

1. Bagaimana latar belakang demografis pada setiap pengguna?
2. Bagaimana kesimpulan kebiasaan pengguna dalam berbelanja?
3. Dari data-data yang sudah dikumpulkan, berapa banyak promosi yang efektif dilakukan untuk setiap pengguna? (Pertanyaan Target)

# Data Wrangling

## Import Data

In [ ]:
trainFeatures = pd.read_csv('data/train_features.csv')
testFeatures = pd.read_csv('data/test_features.csv')
trainLabels = pd.read_csv('data/train_labels.csv')
example = pd.read_csv('data/submission_format.csv')

## Display Data

### Data Train Features

In [ ]:
trainFeatures.head()

In [ ]:
trainFeatures.shape

### Data Train Labels

In [ ]:
trainLabels.head()

In [ ]:
trainLabels.shape

### Data Test Features

In [ ]:
testFeatures.head()

In [ ]:
example.head()

## Concatenate Data Training

Dari data training yang tersedia, perlu dilakukan penggabungan dari kedua data yaitu **Train Features** dan **Train Labels**.

In [ ]:
dataTraining = pd.concat([trainFeatures, trainLabels], axis=1)

In [ ]:
print("Banyak Data Training:", dataTraining.shape[0])
dataTraining.head()

# Data Assessing

## Data Training

### Informasi Umum 

In [ ]:
dataTraining.info()

In [ ]:
dataTraining.describe()

### Cek Nilai Null 

In [ ]:
dataTraining.isnull().sum()

### Cek Nilai Duplikat

In [ ]:
dataTraining.duplicated().sum()

## Data Testing

### Informasi Umum

In [ ]:
testFeatures.info()

In [ ]:
testFeatures.describe()

### Cek Nilai Null

In [ ]:
testFeatures.isna().sum()

### Cek Nilai Duplikat

In [ ]:
testFeatures.duplicated().sum()

# Data Cleaning

### Kolom Pendidikan

In [ ]:
# Cek Nilai Unique Data Training
print(dataTraining['pendidikan'].unique())
# Cek Nilai Unique Data Testing
print(testFeatures['pendidikan'].unique())

Terdapat dua nilai salah yaitu nan dan '5'. Maka perlu dibersihkan pada dua nilai tersebut.

### Olah Data dengan Nilai Unique '5'

In [ ]:
# Ambil contoh nilai pada kolom pendidikan dengan nilai '5' Data Training
dataTraining[dataTraining['pendidikan'] == '5'].head()

In [ ]:
# Ambil contoh nilai pada kolom pendidikan dengan nilai '5' Data Training
testFeatures[testFeatures['pendidikan'] == '5'].head()

In [ ]:
# Data Training
# Mengambil indeks data dengan nilai unique '5'
dropPendidikanTrain = dataTraining[dataTraining['pendidikan'] == '5'].index
# Menghapus nilai dengan indeks tersebut
dataTraining.drop(dropPendidikanTrain, inplace=True)

# Data Testing
# Mengambil indeks data dengan nilai unique '5'
dropPendidikanTest = testFeatures[testFeatures['pendidikan'] == '5'].index
# Menghapus nilai dengan indeks tersebut
testFeatures.drop(dropPendidikanTest, inplace=True)

Saat ini nilai dengan inputan '5' sudah terhapus, karena hanya terdapat 2 data saja maka dapat dikatakan aman untuk dihapus.

Selanjutnya yaitu menangani data dengan inputan nan. Untuk langkah yang diambil yaitu mengecek seberapa banyak nilai nan yang ada. Jika jumlah terbilang sedikit, langkah yang diambil adalah menghapus nilai tersebut (seperti pada inputan '5' sebelumnya). Namun, jika jumlahnya terbilang cukup banyak, maka langkah yang diambil yaitu mengubah setiap nilai nan menjadi nilai modus pada kolom **pendidikan**.

### Olah Data dengan Nilai Unique NaN

In [ ]:
# Data Training
# Mengambil sample data dengan inputan nan
dataNanPendidikanTrain = dataTraining[dataTraining['pendidikan'].isnull()]

# Mengecek Banyak Baris Data
print('Banyak Data dengan inputan nan:', dataNanPendidikanTrain.shape[0], '\n')
print('Dengan sample data sebagai berikut: ')
dataNanPendidikanTrain.sample(10)

In [ ]:
# Data Testing
# Mengambil sample data dengan inputan nan
dataNanPendidikanTest = testFeatures[testFeatures['pendidikan'].isnull()]

# Mengecek Banyak Baris Data
print('Banyak Data dengan inputan nan:', dataNanPendidikanTest.shape[0], '\n')
print('Dengan sample data sebagai berikut: ')
dataNanPendidikanTest.sample(10)

Dikarenakan jumlah dari data nan cukup banyak, maka akan dilakukan pengubahan isi dengan data modus pada kolom **pendidikan**.

In [ ]:
# Data Training
# Mengambil jumlah isi terbanyak pada kolom pendidikan
modePendidikanTrain = dataTraining['pendidikan'].mode()[0]
print('Nilai dengan modus terbanyak adalah', modePendidikanTrain)
# Mengubah nilai nan menjadi nilai modus
dataTraining['pendidikan'] = dataTraining['pendidikan'].fillna(modePendidikanTrain)

# Data Testing
modePendidikanTest = testFeatures['pendidikan'].mode()[0]
print('Nilai dengan modus terbanyak adalah', modePendidikanTest)
# Mengubah nilai nan menjadi nilai modus
testFeatures['pendidikan'] = testFeatures['pendidikan'].fillna(modePendidikanTest)

In [ ]:
# Data Train
# Cek ulang nilai unique
print(dataTraining['pendidikan'].unique())

# Data Test
# Cek ulang nilai unique
print(testFeatures['pendidikan'].unique())

Pada saat ini, kondisi kolom **pendidikan** sudah normal.

## Kolom Status Pernikahan

In [ ]:
# Data Training
# Cek Nilai Unique
print(dataTraining['status_pernikahan'].unique())

# Data Training
# Cek Nilai Unique
print(testFeatures['status_pernikahan'].unique())

Mirip dengan kolom Pendidikan, terdapat dua nilai tidak sesuai pada kolom **Status Pernikahan** yaitu '5' dan nan. Maka perlu dilakukan proses yang sama dengan kolom Pendidikan yaitu meninjau setiap invalid value yang ada untuk dilakukan langkah lebih lanjut.

### Olah Data dengan Nilai Unique '5'

In [ ]:
# Data Training
# Mengambil panjang data dengan nilai unique '5'
print(dataTraining[dataTraining['status_pernikahan'] == '5'].shape)

# Data Training
# Mengambil panjang data dengan nilai unique '5'
print(testFeatures[testFeatures['status_pernikahan'] == '5'].shape)

Dikarenakan hanya terdapat satu nilai saja dengan value '5', maka dapat dihapus saja karena tidak terlalu berpengaruh terhadap keseluruhan data.

In [ ]:
# Data Training
# Mengambil data dengan nilai unique '5'
dropStatusNikahTrain = dataTraining[dataTraining['status_pernikahan'] == '5'].index
# Menghapus data
dataTraining.drop(dropStatusNikahTrain, inplace=True)

# Data Testing
# Mengambil data dengan nilai unique '5'
dropStatusNikahTest = testFeatures[testFeatures['status_pernikahan'] == '5'].index
# Menghapus data
testFeatures.drop(dropStatusNikahTest, inplace=True)


### Olah Data dengan Nilai Unique NaN

In [ ]:
# Data Training
# Mengambil sample data dengan inputan nan
dataNanPernikahanTrain = dataTraining[dataTraining['status_pernikahan'].isnull()]

# Mengecek Banyak Baris Data
print('Banyak Data dengan inputan nan:', dataNanPernikahanTrain.shape[0], '\n')

# Data Testing
# Mengambil sample data dengan inputan nan
dataNanPernikahanTest = testFeatures[testFeatures['status_pernikahan'].isnull()]

# Mengecek Banyak Baris Data
print('Banyak Data dengan inputan nan:', dataNanPernikahanTest.shape[0], '\n')

Dikarenakan jumlah dari data nan cukup banyak, maka akan dilakukan pengubahan isi data nan dengan analisis lebih lanjut pada kolom-kolom yang berpotensi memiliki korelasi dengan kolom **status pernikahan**. 

Yaitu dengan melihat kolom **jumlah anak balita** dan **jumlah anak remaja** dengan asumsi jika memiliki nilai lebih dari 0 dari kedua kolom tersebut, maka akan diubah menjadi "Menikah".

In [ ]:
# Isi Data Train dan Test
listDf = [dataTraining, testFeatures]

# Buat Fungsi
def ubahPernikahan(row):
    if row['status_pernikahan'] == 'kosong':
        if row['jumlah_anak_balita'] > 0 or row['jumlah_anak_remaja'] > 0:
            return 'Menikah'
        else:
            return 'Sendiri'
    else:
        return row['status_pernikahan']
    
for list in listDf:
    # Ubah NaN menjadi String Terlebih Dahulu
    list['status_pernikahan'].fillna('kosong', inplace=True)

    list['status_pernikahan'] = list.apply(ubahPernikahan, axis=1)

In [ ]:
# Data Training
print(dataTraining['status_pernikahan'].unique())

# Data Testing
print(testFeatures['status_pernikahan'].unique())

Saat ini nilai unique pada kolom **status pernikahan** sudah normal dan tidak ada nilai nan.

## Kolom Pendapatan

In [ ]:
listDf = [dataTraining, testFeatures]

### Cek Nilai Statistik 

In [ ]:
# Data Training
listDf[0].describe()

In [ ]:
# Data Testing
listDf[1].describe()

### Pengecekan Nilai Null

Dari hasil analisis sebelumnya, terlihat bahwa terdapat nilai null pada kolom ini.

In [ ]:
# Data Training
nullPendapatanTrain = listDf[0][listDf[0]['pendapatan'].isnull()]
nullPendapatanTrain.head(10)

In [ ]:
# Data Testing
nullPendapatanTest = listDf[1][listDf[1]['pendapatan'].isnull()]
nullPendapatanTest.head(10)

Yang dilakukan adalah menggunakan interpolasi polinomial untuk menutup setiap nilai NaN yang ada dengan titik terdekat data ke data lainnya menggunakan garis polinomial berorde 2.

In [ ]:
for data in listDf:
    # Data x dan y yang tidak null
    x_known = data.index[~data['pendapatan'].isnull()]
    y_known = data.loc[x_known, 'pendapatan']

    # Buat objek interpolasi polinomial orde rendah (misalnya, orde 2)
    poly_interp = Polynomial.fit(x_known, y_known, deg=2)

    # Memasang indeks titik data yang akan diisi (NaN)
    fill_indices = data.index[data['pendapatan'].isnull()]

    # Isi nilai-nilai NaN dengan hasil interpolasi polinomial
    data.loc[fill_indices, 'pendapatan'] = poly_interp(fill_indices)


In [ ]:
# Data Training
print(listDf[0]['pendapatan'].isna().sum())

# Data Testing
print(listDf[1]['pendapatan'].isna().sum())

Pada saat ini sudah tidak terdapat nilai NaN lagi pada kolom **pendapatan**, sehingga dapat dilanjutkan pada proses analisis selanjutnya.

## Kolom Jumlah Anak Balita

### Cek Nilai Unique

In [ ]:
# Data Training
print(listDf[0]['jumlah_anak_balita'].unique())

# Data Test
print(listDf[1]['jumlah_anak_balita'].unique())

Pada pengecekan nilai unique di atas, ditemukan nilai NaN yang perlu diganti dengan nilai lain.

### Hapus Nilai Null

In [ ]:
for data in listDf:
    data['jumlah_anak_balita'].fillna(0, inplace=True)
    data['jumlah_anak_balita'] = data['jumlah_anak_balita'].astype(int)
    print(data['jumlah_anak_balita'].dtype)

In [ ]:
# Data Training
print(listDf[0]['jumlah_anak_balita'].unique())

# Data Test
print(listDf[1]['jumlah_anak_balita'].unique())

Pada proses ini, langkah yang diambil adalah mengubah nilai NaN menjadi **0** dengan asumsi setiap nilai NaN diartikan baris data penduduk tersebut memiliki jumlah anak balita sebanyak **0**.

## Kolom Jumlah Anak Remaja

### Cek Nilai Unique

In [ ]:
# Data Training
print(listDf[0]['jumlah_anak_remaja'].unique())

# Data Test
print(listDf[1]['jumlah_anak_remaja'].unique())

Sama seperti pada kasus kolom *jumlah anak balita*, terdapat nilai NaN pada kolom *jumlah anak remaja* yang perlu diubah dengan nilai lainnya.

### Hapus Nilai Null

In [ ]:
for data in listDf:
    data['jumlah_anak_remaja'].fillna(0, inplace=True)
    data['jumlah_anak_remaja'] = data['jumlah_anak_balita'].astype(int)
    print(data['jumlah_anak_remaja'].dtype)

In [ ]:
# Data Training
print(listDf[0]['jumlah_anak_remaja'].unique())

# Data Test
print(listDf[1]['jumlah_anak_remaja'].unique())

Hasil pada kolom *jumlah anak remaja* juga mirip dengan hasil kolom *jumlah anak balita*, yaitu dengan mengubah nilai NaN dengan nilai **0**. Sebagai asumsi bahwa nilai NaN berarti baris data penduduk tersebut tidak memiliki anak remaja.

## Kolom Terakhir Belanja

### Cek Nilai Unique

In [ ]:
# Data Training
listDf[0]['terakhir_belanja'].unique()

In [ ]:
# Data Testing
listDf[1]['terakhir_belanja'].unique()

Pada pengecekan nilai unique, hasilnya terdapat nilai NaN yang terkandung didalam kolom *terakhir belanja*. Maka perlu dilakukan pengecekan batas minimum dan maksimum data serta nilai statistik lainnya seperti rata-rata untuk mengubah nilai NaN ini dengan sajian nilai Unique yang lebih rapi.

### Cek Nilai Min dan Max

In [ ]:
for i, data in enumerate(listDf):
    nilaiMax = round(data.terakhir_belanja.max())
    nilaiMin = round(data.terakhir_belanja.min())

    if i == 0:
        namaDf = 'training'
    else:
        namaDf = 'testing'
    print(f'Nilai maksimal dari dataset {namaDf} kolom terakhir belanja adalah {nilaiMax}')
    print(f'Nilai minimal dari dataset {namaDf} kolom terakhir belanja adalah {nilaiMin}')
    print()

Dikarenakan tidak adanya acuan data tambahan dari kolom *terakhir belanja*, maka langkah yang diambil adalah mengubah setiap nilai NaN menjadi nilai rata-rata.

### Ubah Nilai Null dengan Rata-Rata

In [ ]:
for i, data in enumerate(listDf):
    # mengambil nilai mean setiap dataset
    mean = data['terakhir_belanja'].mean()
    
    if i == 0:
        namaDf = 'training'
    else:
        namaDf = 'testing'
    
    # Menampilkan angka mean
    print(f'Nilai rata-rata dataset {namaDf} adalah {mean}')
    
    # Eksekusi pengisian nilai NaN dengan nilai mean
    data['terakhir_belanja'] = data['terakhir_belanja'].fillna(mean)
    data['terakhir_belanja'] = data['terakhir_belanja'].astype(int)
    print(data['terakhir_belanja'].dtype)   

Setelah dilakukan pengisian nilai nan dengan nilai rata-rata (mean), selanjutnya dilakukan pengecekan ulang pada setiap nilai yang ada

In [ ]:
for i, data in enumerate(listDf):
    # Mengurutkan nilai unique
    nilaiTerurut = np.sort(data['terakhir_belanja'].unique())
    
    if i == 0:
        namaDf = 'training'
    else:
        namaDf = 'testing'
    
    # Tampilkan nilai unique terurut
    print(f'Dataset {namaDf}:')
    print(nilaiTerurut)
    print()

## Kolom Belanja

Pada kolom **Belanja**, juga masih terdapat nilai NaN yang harus dipenuhi. Untuk langkah yang diambil yaitu mengisi dengan menggunakan nilai-nilai dari hasil prediksi menggunakan metode **Polynomial**.

Hal ini dimaksudkan mengisi nilai-nilai NaN dengan nilai-nilai hasil prediksi menggunakan garis polynomial, lalu mengambil nilai yang bersinggungan atau berdekatan dengan nilai yang bersebelahan.

### Eksekusi Polynomial Pengisian Nilai Null

In [ ]:
# Mendefinisikan nama-nama kolom
kolBelanja = ['belanja_buah', 'belanja_daging', 'belanja_ikan', 'belanja_kue']

In [ ]:
for i, data in enumerate(listDf):
    # Kondisi untuk nama dataset yang ditampilkan
    if i == 0:
        namaDf = 'training'
    else:
        namaDf = 'testing'
    # Menampilkan nama dataset 
    print(f'Banyak data yang masih null pada dataset {namaDf}: ')
    
    for col in kolBelanja:
        # Data x dan y yang tidak null
        x_known = data.index[~data[col].isnull()]
        y_known = data.loc[x_known, col]

        # Buat objek interpolasi polinomial orde rendah (misalnya, orde 2)
        poly_interp = Polynomial.fit(x_known, y_known, deg=2)

        # Memasang indeks titik data yang akan diisi (NaN)
        fill_indices = data.index[data[col].isnull()]

        # Isi nilai-nilai NaN dengan hasil interpolasi polinomial
        data.loc[fill_indices, col] = poly_interp(fill_indices)
        # Membuat kondisi untuk nama kolom yang ditampilkan
        if col == 'belanja_buah':
            colName = 'Belanja Buah'
        elif col == 'belanja_daging':
            colName = 'Belanja Daging'
        elif col == 'belanja_ikan':
            colName = 'Belanja Ikan'
        else:
            colName = 'Belanja Kue'
        # Menampilkan nama kolom dan jumlah nilai nullnya
        print(f'{colName}: {data[col].isna().sum()}')
    print()

Dari hasil eksekusi di atas, sudah **tidak ditemukan nilai null** baik di Dataset Training maupun Dataset Testing di setiap kolom belanja.

## Kolom Pembelian Diskon

Pada kolom **Pembelian Diskon**, dilakukan pengisian nilai NaN dengan nilai mean pada setiap segmentasi pendapatan yang diambil dari kolom **Pendapatan**. Namun sebelum dieksekusi, perlu menyiapkan kolom baru bernama **Segmentasi Pendapatan** yang berisi nilai segmentasi pendapatan, diambil dari angka kuartil di kolom **Pendapatan**.

In [ ]:
# Buat label segmentasi
labelSegment = ['Rendah', 'Sedang', 'Tinggi', 'Sangat Tinggi']

for data in listDf:
      # Mengambil nilai kuartil pendapatan
      kuartilPendapatan = data['pendapatan'].quantile([0.25, 0.5, 0.75])
      # Membuat kolom segmentasi pendapatna
      data['segmentasi_pendapatan'] = pd.cut(data['pendapatan'], 
                                                      bins=[0, kuartilPendapatan.iloc[0], kuartilPendapatan.iloc[1], 
                                                            kuartilPendapatan.iloc[2], data['pendapatan'].max()],
                                                      labels=labelSegment)
      
      for segmen in data['segmentasi_pendapatan'].unique():
            # Filter DataFrame berdasarkan segmen pendapatan
            df_segmen = data[data['segmentasi_pendapatan'] == segmen]
            
            # Mengambil nilai mean dari kolom 'pembelian_diskon' pada segmen saat ini
            mayorSegmen = df_segmen['pembelian_diskon'].mean()

            # Mengisi nilai null di kolom 'pembelian_diskon' berdasarkan segmentasi pendapatan dengan nilai modus
            data.loc[data['segmentasi_pendapatan'] == segmen, 'pembelian_diskon'] = data.loc[data['segmentasi_pendapatan'] == segmen, 'pembelian_diskon'].fillna(mayorSegmen)

In [ ]:
# Cek banyak nilai null
# Data Training
print(dataTraining.pembelian_diskon.isna().sum())
# Data Training
print(testFeatures.pembelian_diskon.isna().sum())

Setelah dilakukan proses pengisian nilai NaN dengan nilai mean, terlihat bahwa sudah tidak ada nilai null baik di **Data Training** dan **Data Testing** di **Kolom Pembelian Diskon**.

## Kolom Pembelian Web

Pada kolom **Pembelian Web**, untuk mengisi nilai NaN akan dilakukan dengan cara yang sama ketika mengisi nilai NaN di kolom **Pembelian Diskon**. Hanya saja tidak ada proses pembuatan kolom **Segmentasi Pendapatan**. Berikut adalah kode eksekusinya:

In [ ]:
for data in listDf:
      for segmen in data['segmentasi_pendapatan'].unique():
            # Filter DataFrame berdasarkan segmen pendapatan
            df_segmen = data[data['segmentasi_pendapatan'] == segmen]
            
            # Mengambil nilai mean dari kolom 'pembelian_diskon' pada segmen saat ini
            mayorSegmen = df_segmen['pembelian_web'].mean()

            # Mengisi nilai null di kolom 'pembelian_diskon' berdasarkan segmentasi pendapatan dengan nilai modus
            data.loc[data['segmentasi_pendapatan'] == segmen, 'pembelian_web'] = data.loc[data['segmentasi_pendapatan'] == segmen, 'pembelian_web'].fillna(mayorSegmen)

In [ ]:
# Cek banyak nilai null
# Data Training
print(dataTraining.pembelian_web.isna().sum())
# Data Training
print(testFeatures.pembelian_web.isna().sum())

Setelah dilakukan proses pengisian nilai NaN dengan nilai mean, terlihat bahwa sudah tidak ada nilai null baik di **Data Training** dan **Data Testing** di **Kolom Pembelian Web**.

## Kolom Pembelian Toko

Pada kolom **Pembelian Web**, untuk mengisi nilai NaN akan dilakukan dengan cara yang sama ketika mengisi nilai NaN di kolom **Pembelian Diskon** dan **Pembelian Web**. Hanya saja tidak ada proses pembuatan kolom **Segmentasi Pendapatan**. Berikut adalah kode eksekusinya:

In [ ]:
for data in listDf:
      for segmen in data['segmentasi_pendapatan'].unique():
            # Filter DataFrame berdasarkan segmen pendapatan
            df_segmen = data[data['segmentasi_pendapatan'] == segmen]
            
            # Mengambil nilai mean dari kolom 'pembelian_diskon' pada segmen saat ini
            mayorSegmen = df_segmen['pembelian_toko'].mean()

            # Mengisi nilai null di kolom 'pembelian_diskon' berdasarkan segmentasi pendapatan dengan nilai modus
            data.loc[data['segmentasi_pendapatan'] == segmen, 'pembelian_toko'] = data.loc[data['segmentasi_pendapatan'] == segmen, 'pembelian_toko'].fillna(mayorSegmen)

In [ ]:
# Cek banyak nilai null
# Data Training
print(dataTraining.pembelian_toko.isna().sum())
# Data Training
print(testFeatures.pembelian_toko.isna().sum())

Setelah dilakukan proses pengisian nilai NaN dengan nilai mean, terlihat bahwa sudah tidak ada nilai null baik di **Data Training** dan **Data Testing** di **Kolom Pembelian Toko**.

## Kolom Keluhan

Pada kolom **Keluhan**, dilakukan pengisian nilai NaN dengan menggunakan angka 0. Hal ini mengasumsikan bahwa nilai NaN berarti pelanggan tersebut tidak memiliki keluhan pada proses jual beli.

In [ ]:
for data in listDf:
    # Mengisi dengan angka 0 setiap dataset
    data['keluhan'].fillna(0, inplace=True)

In [ ]:
# Cek banyak nilai null
# Data Training
print(dataTraining.keluhan.isna().sum())
# Data Training
print(testFeatures.keluhan.isna().sum())

## Kolom Tanggal Menjadi Anggota

Pada kolom **Tanggal Menjadi Anggota**, pengisian nilai NaN dengan teks "Unknown". Dengan asumsi bahwa pelanggan belum menjadi anggota.

In [ ]:
for data in listDf:
    data['tanggal_menjadi_anggota'].fillna("Unknown", inplace=True)

In [ ]:
# Cek banyak nilai null
# Data Training
print(dataTraining.tanggal_menjadi_anggota.isna().sum())
# Data Training
print(testFeatures.tanggal_menjadi_anggota.isna().sum())

## Kolom Segmentasi Pendapatan

In [ ]:
# Cek banyak nilai null
# Data Training
print(dataTraining.segmentasi_pendapatan.isna().sum())
# Data Training
print(testFeatures.segmentasi_pendapatan.isna().sum())

Sebab tidak terdapat nilai NaN, maka tidak perlu adanya penghapusan nilai NaN di kolom ini.

## Pembulatan Dataset

Langkah ini dilakukan agar data numerik pada setiap dataset menjadi bulat.

In [ ]:
# Membulatkan dataset (kolom numerik)
for data in listDf:
    data = round(data)

### Data Training

In [ ]:
dataTraining.head()

### Data Testing

In [ ]:
dataTesting = testFeatures.copy()
dataTesting.head()

# Exploratory Data Analysis (EDA)

## Analisis Invalid Value

## Pertanyaan Analisis
Dari pertanyaan-pertanyaan bisnis yang sudah didefinisikan, berikut adalah pertanyaan-pertanyaan guna menganalisis setiap pertanyaan bisnis yang ada:

1. Rentang usia berapa saja pelanggan yang terdata pada dataset?
2. Berapa porsi berdasarkan pendidikan terakhir pelanggan?
3. Status pernikahan apa saja pelanggan yang terdata pada dataset?
4. Berapa rata-rata pelanggan yang sudah memiliki anak dan berapa jumlah rata-rata anak yang dimiliki?
5. Berapa rata-rata pendapatan dari segmentasi pendapatan yang ada?
6. Dari segmentasi pendapatan, bagaimana analisis kebiasaan pelanggan dalam belanja?
7. Bagaimana hubungan keluhan pelanggan berdasarkan cara pembelian yang dilakukan?
8. Bagaimana distribusi pendaftaran anggota dari hasil analisis statistik dan demografis yang sudah dilakukan?
9. Bagaimana hubungan antara kepemilikan anak dan status pernikahan terhadap belanja?

## Analisis Outliers

Sebelum melakukan analisis lebih lanjut, dilakukan analisis outliers untuk menghilangkan nilai-nilai diluar rentang yang dapat mempengaruhi hasil dari analisis.

### Cek Nilai Outliers dengan Box Plot

In [ ]:
# Data Training
numFeatures = dataTraining.select_dtypes(include='number').columns

# Tentukan ukuran kanvas subplot
numPlots = len(numFeatures)
numRows = numPlots // 2 + numPlots % 2  # Hitung jumlah baris
fig, axes = plt.subplots(nrows=numRows, ncols=2, figsize=(12, 20), 
                         gridspec_kw={'height_ratios': [1]*numRows})  # Set tinggi setiap baris sama

# Looping melalui kolom numerik
for i, column in enumerate(numFeatures):
    row = i // 2
    col = i % 2
    sns.boxplot(x=dataTraining[column], ax=axes[row, col])
    axes[row, col].set_title(column)

# Menghapus subplot yang tidak digunakan jika jumlah kolom ganjil
if numPlots % 2 != 0:
    fig.delaxes(axes[numRows-1, 1])

plt.tight_layout()
plt.show()

In [ ]:
# Data Testing
numFeatures = dataTesting.select_dtypes(include='number').columns

# Tentukan ukuran kanvas subplot
numPlots = len(numFeatures)
numRows = numPlots // 2 + numPlots % 2  # Hitung jumlah baris
fig, axes = plt.subplots(nrows=numRows, ncols=2, figsize=(12, 20), 
                         gridspec_kw={'height_ratios': [1]*numRows})  # Set tinggi setiap baris sama

# Looping melalui kolom numerik
for i, column in enumerate(numFeatures):
    row = i // 2
    col = i % 2
    sns.boxplot(x=dataTesting[column], ax=axes[row, col])
    axes[row, col].set_title(column)

# Menghapus subplot yang tidak digunakan jika jumlah kolom ganjil
if numPlots % 2 != 0:
    fig.delaxes(axes[numRows-1, 1])

plt.tight_layout()
plt.show()

Dari hasil analisis banyak outliers di atas, terlihat bahwa ada beberapa kolom yang masih terdapat nilai outliers yang perlu ditangani. Namun juga perlu adanya seleksi lagi kolom-kolom yang perlu ditangani dan juga yang tidak. 

Untuk kolom yang tidak perlu yaitu kolom-kolom numerik dengan nilai terbilang sedikit variasi nilainya.

### Hapus Nilai Outliers

In [ ]:
# Data Training

ignoredColumn = ['keluhan', 'jumlah_anak_balita', 'jumlah_anak_remaja', 'tahun_kelahiran']
interpolateColumn = ['belanja_buah', 'belanja_ikan', 'belanja_daging', 'belanja_kue']

# Fungsi untuk menghapus outlier menggunakan pendekatan IQR
def replace_outliers_with_interpolation(df):
    cleaned_data = df.copy()  # Salin dataframe asli untuk dimodifikasi
    
    # Loop melalui setiap kolom numerik
    for column in df.select_dtypes(include='number').columns:
        if column not in ignoredColumn:  # Lewati kolom yang diabaikan
            Q1 = df[column].quantile(0.25)
            Q3 = df[column].quantile(0.75)
            IQR = Q3 - Q1
            lower_bound = Q1 - 1.5 * IQR
            upper_bound = Q3 + 1.5 * IQR
            
            if column in interpolateColumn:
                # Interpolasi nilai outliers
                outliers = df[(df[column] < lower_bound) | (df[column] > upper_bound)]
                interpolated_values = (outliers[column] - lower_bound) / (upper_bound - lower_bound) * (Q3 - Q1) + Q1
                cleaned_data.loc[outliers.index, column] = interpolated_values
            cleaned_data[column] = cleaned_data[column].apply(lambda x: lower_bound if x < lower_bound else (upper_bound if x > upper_bound else x))
    return cleaned_data

# Hapus outlier menggunakan pendekatan IQR untuk setiap kolom (kecuali yang diabaikan)
cleanedTraining = replace_outliers_with_interpolation(dataTraining)

In [ ]:
# Data Testing

ignoredColumn = ['keluhan', 'jumlah_anak_balita', 'jumlah_anak_remaja', 'tahun_kelahiran', 'ID']
interpolateColumn = ['belanja_buah', 'belanja_ikan', 'belanja_daging', 'belanja_kue']

# Fungsi untuk menghapus outlier menggunakan pendekatan IQR
def replace_outliers_with_interpolation(df):
    cleaned_data = df.copy()  # Salin dataframe asli untuk dimodifikasi
    
    # Loop melalui setiap kolom numerik
    for column in df.select_dtypes(include='number').columns:
        if column not in ignoredColumn:  # Lewati kolom yang diabaikan
            Q1 = df[column].quantile(0.25)
            Q3 = df[column].quantile(0.75)
            IQR = Q3 - Q1
            lower_bound = Q1 - 1.5 * IQR
            upper_bound = Q3 + 1.5 * IQR
            
            if column in interpolateColumn:
                # Interpolasi nilai outliers
                outliers = df[(df[column] < lower_bound) | (df[column] > upper_bound)]
                interpolated_values = (outliers[column] - lower_bound) / (upper_bound - lower_bound) * (Q3 - Q1) + Q1
                cleaned_data.loc[outliers.index, column] = interpolated_values
            cleaned_data[column] = cleaned_data[column].apply(lambda x: lower_bound if x < lower_bound else (upper_bound if x > upper_bound else x))
    return cleaned_data

# Hapus outlier menggunakan pendekatan IQR untuk setiap kolom (kecuali yang diabaikan)
cleanedTesting = replace_outliers_with_interpolation(dataTesting)

### Cek Ulang Nilai Outliers

In [ ]:
# Data Training

numFeatures = cleanedTraining.select_dtypes(include='number').columns

# Tentukan ukuran kanvas subplot
numPlots = len(numFeatures)
numRows = numPlots // 2 + numPlots % 2  # Hitung jumlah baris
fig, axes = plt.subplots(nrows=numRows, ncols=2, figsize=(12, 20), 
                         gridspec_kw={'height_ratios': [1]*numRows})  # Set tinggi setiap baris sama

# Looping melalui kolom numerik
for i, column in enumerate(numFeatures):
    row = i // 2
    col = i % 2
    sns.boxplot(x=cleanedTraining[column], ax=axes[row, col])
    axes[row, col].set_title(column)

# Menghapus subplot yang tidak digunakan jika jumlah kolom ganjil
if numPlots % 2 != 0:
    fig.delaxes(axes[numRows-1, 1])

plt.tight_layout()
plt.show()

In [ ]:
# Data Testing

numFeatures = cleanedTesting.select_dtypes(include='number').columns

# Tentukan ukuran kanvas subplot
numPlots = len(numFeatures)
numRows = numPlots // 2 + numPlots % 2  # Hitung jumlah baris
fig, axes = plt.subplots(nrows=numRows, ncols=2, figsize=(12, 20), 
                         gridspec_kw={'height_ratios': [1]*numRows})  # Set tinggi setiap baris sama

# Looping melalui kolom numerik
for i, column in enumerate(numFeatures):
    row = i // 2
    col = i % 2
    sns.boxplot(x=cleanedTesting[column], ax=axes[row, col])
    axes[row, col].set_title(column)

# Menghapus subplot yang tidak digunakan jika jumlah kolom ganjil
if numPlots % 2 != 0:
    fig.delaxes(axes[numRows-1, 1])

plt.tight_layout()
plt.show()

Setelah dilakukan proses penghapusan outliers, saat ini sudah tidak ada outliers pada setiap kolom di data training.

## Jawab Pertanyaan Analisis, dan Analisis Invalid Value Lanjut

Pada sesi ini, menjawab pertanyaan analisis dengan berbagai langkah analisis disertai dengan Analisis Invalid Value Lanjut. Untuk proses analisis yang digunakan hanyalah dataset **Training**. Sebab terdapat perbedaan pola dataset.

### Pertanyaan 1

Pada pertanyaan 1, perlu melakukan analisa pada kolom **Tahun Kelahiran** untuk mengetahui usia berapa saja setiap pengguna yang terdata pada dataset.

In [ ]:
# Mengambil nilai tahun sekarang
yearNow = datetime.now().year
# Mengurangi tahun kelahiran dengan tahun sekarang, masukkan dalam kolom baru
# Data Training
cleanedTraining['usia'] = yearNow - cleanedTraining['tahun_kelahiran']
# Data Training
cleanedTesting['usia'] = yearNow - cleanedTesting['tahun_kelahiran']

Setelah mendapatkan angka usia pada setiap pengguna, selanjutnya adalah membuat rentang usia.

In [ ]:
# Membuat batas rentang usia
limitAges = [0, 30, 70, float('inf')]

# Membuat label rentang usia
labelAges = ['Remaja', 'Dewasa', 'Lanjut Usia']

# Membuat kolom baru rentang usia
# Data Training
cleanedTraining['rentang_usia'] = pd.cut(cleanedTraining['usia'], bins=limitAges, labels=labelAges)
# Data Testing
cleanedTesting['rentang_usia'] = pd.cut(cleanedTesting['usia'], bins=limitAges, labels=labelAges)

Setelah dilakukan pembuatan rentang usia, maka perlu dilakukan proses validasi pada nilai-nilai yang ada dan berkaitan dengan pertanyaan 1.

In [ ]:
print('Usia paling muda adalah', cleanedTraining.usia.min())
print('Usia paling tua adalah', cleanedTraining.usia.max())

Disini terdeteksi terdapat ketidaktepatan pada data yang sudah ada, yaitu adanya usia yang tidak logis. Maka perlu dilakukan analisis lebih lanjut pada data kolom Tahun **Kelahiran**, **Usia**, dan **Rentang Usia**.

In [ ]:
# Grouping berdasarkan rentang usia
grpAgeRange = cleanedTraining[cleanedTraining['rentang_usia'] == 'Lanjut Usia']

# Mengurutkan setiap nilai unik di kolom usia untuk menentukan batas maksimal usia
print(np.sort(grpAgeRange['usia'].unique()))

# Menampilkan banyak data di atas batas maksimal
print("Banyak data di atas batas maksimal", len(cleanedTraining[cleanedTraining['usia'] > 84]))

Dari hasil analisis di atas, dari penentuan batas maksimal usia lansia adalah **84** dengan banyak data di atas batas maksimal (Invalid Value) adalah **16**. Maka dapat dilakukan penghapusan data di atas batas maksimal.

In [ ]:
cleanedTraining = cleanedTraining[cleanedTraining['usia'] <= 84]

Setelah dilakukan pembersihan, dapat disimpulkan rentang usia berdasarkan usia pengguna adalah sebagai berikut:

In [ ]:
grpAges = cleanedTraining.groupby('rentang_usia').agg({
    'usia': ['count', 'min', 'max']
})

for rentang, nilai in grpAges.iterrows():
    print(f"Pada rentang usia {rentang} yaitu usia {nilai['usia']['min']}-{nilai['usia']['max']}, terdapat {nilai['usia']['count']} pengguna.")

In [ ]:
# Plotting bar chart
plt.figure(figsize=(13, 6))
bars = grpAges['usia', 'count'].plot(kind='barh')
plt.xlabel('Jumlah Pengguna')
plt.ylabel('Rentang Usia')
plt.title('Jumlah Pengguna berdasarkan Rentang Usia')
plt.legend().remove()  # Menghapus legend

# Menambahkan persentase pada setiap bar
for bar in bars.patches:
    plt.text(bar.get_width(), bar.get_y() + bar.get_height() / 2,
             f'{bar.get_width() / sum(grpAges["usia", "count"]) * 100:.2f}%',
             va='center')

plt.show()

Dari hasil visualisasi data pada analisis univarian di atas dapat disimpulkan bahwa pengguna terbanyak yang terdata adalah golongan rentang usia **Dewasa**.

### Pertanyaan 2

Pada sesi ini akan menjawab pertanyaan 2, dengan menganalisa nilai-nilai yang ada di kolom **Pendidikan**. Tujuannya yaitu untuk mengetahui banyak porsi presentase setiap jenis pendidikan terakhir.

In [ ]:
# Nilai unique kolom pendidikan
cleanedTraining.pendidikan.unique()

In [ ]:
count = cleanedTraining['pendidikan'].value_counts()
df = pd.DataFrame({
    'Jumlah sampel':count, 
})
print(df)

# Plotting bar chart vertikal
plt.figure(figsize=(13, 6))
bars = count.plot(kind='bar', rot=0)
plt.xlabel('Pendidikan Terakhir')
plt.ylabel('Jumlah Pengguna')
plt.title('Jumlah Pengguna berdasarkan Pendidikan Terakhir')
plt.legend().remove()  # Menghapus legend

# Menambahkan persentase pada setiap bar
for bar in bars.patches:
    plt.text(bar.get_x() + bar.get_width() / 2, bar.get_height(),
             f'{bar.get_height() / sum(count) * 100:.2f}%',
             ha='center', va='bottom')

plt.show()

Dari hasil visualisasi data pada analisis univarian di atas dapat disimpulkan bahwa pengguna mayoritas memiliki pendidikan terakhir **Sarjana**.

### Pertanyaan 3

Pada sesi ini akan dilakukan analisis latar belakang pengguna berdasarkan status pernikahan

In [ ]:
# Nilai unique status pernikahan
cleanedTraining.status_pernikahan.unique()

In [ ]:
count = cleanedTraining['status_pernikahan'].value_counts()
df = pd.DataFrame({
    'Jumlah sampel':count, 
})
print(df)

# Plotting bar chart vertikal
plt.figure(figsize=(13, 6))
bars = count.plot(kind='bar', rot=0)
plt.xlabel('Status Pernikahan')
plt.ylabel('Jumlah Pengguna')
plt.title('Jumlah Pengguna Berdasarkan Status Pernikahan')
plt.legend().remove()  # Menghapus legend

# Menambahkan persentase pada setiap bar
for bar in bars.patches:
    plt.text(bar.get_x() + bar.get_width() / 2, bar.get_height(),
             f'{bar.get_height() / sum(count) * 100:.2f}%',
             ha='center', va='bottom')

plt.show()

Dari hasil visualisasi data pada analisis univarian di atas dapat disimpulkan bahwa pengguna mayoritas memiliki status pernikahan **Rencana Menikah**, yang berbeda tipis dengan status pernikahan **Menikah**.

### Pertanyaan 4

Pada sesi ini akan dilakukan analisis latar belakang pengguna berdasarkan kepemilikan anak, dengan menganalisis kolom **Jumlah Anak Balita** dan **Jumlah Anak Remaja**

In [ ]:
# Buat Kolom Baru, yaitu memiliki anak
def haveChildFunct(df):
    for index, row in df.iterrows():
        # Kondisi penentuan nilai kepemilikan anak
        if row['jumlah_anak_balita'] > 0 or row['jumlah_anak_remaja'] > 0:
            df.at[index, 'memiliki_anak'] = 'Memiliki'
        else:
            df.at[index, 'memiliki_anak'] = 'Tidak Memiliki'

# Data Training
haveChildFunct(cleanedTraining)

# Data Training
haveChildFunct(cleanedTesting)

Saat ini dataframe utama sudah memiliki kolom kepemilikan anak. Selanjutnya adalah melakukan agregasi data untuk memvalidasi data yang ada.

In [ ]:
# Hitung jumlah anak berdasarkan status pernikahan (validasi)
countWed = cleanedTraining.groupby(['status_pernikahan', 'memiliki_anak'])['memiliki_anak'].count()
countWed

Pada hasil di atas, nampaknya terdapat kesalahan data yaitu pada kategori **Sendiri** dan **Rencana Menikah** yang tercatat **memiliki anak**. Untuk menghindari mispersepsi, penanganan akan dilakukan pada dataset dengan cara mengubah nilai kepemilikan anak, serta menjalankan proses pengisian kolom **Memiliki Anak** secara ulang.

In [ ]:
# Filter DataFrame untuk status pernikahan "Sendiri"
beforeMarried = cleanedTraining[(cleanedTraining['status_pernikahan'] == 'Sendiri') | (cleanedTraining['status_pernikahan'] == 'Rencana Menikah')]

# Ubah nilai kolom 'jumlah_anak_balita', 'jumlah_anak_remaja', dan 'memiliki_anak' untuk baris-baris tersebut
cleanedTraining.loc[beforeMarried.index, 'jumlah_anak_balita'] = 0
cleanedTraining.loc[beforeMarried.index, 'jumlah_anak_remaja'] = 0
cleanedTraining.loc[beforeMarried.index, 'memiliki_anak'] = 'Tidak Memiliki'


In [ ]:
# Cek ulang Nilai Count
# Hitung jumlah anak berdasarkan status pernikahan (validasi)
sendiriRow = cleanedTraining[(cleanedTraining['status_pernikahan'] == 'Rencana Menikah') & (cleanedTraining['memiliki_anak'] == 'Memiliki')]
sendiriRow.shape[0]

Setelah dipastikan sudah tidak ada nilai invalid yang ada, maka selanjutnya dilakukan proses visualisasi data.

In [ ]:
# Hitung jumlah masing-masing nilai unik dalam kolom 'memiliki_anak'
count = cleanedTraining['memiliki_anak'].value_counts()

# Plotting pie chart
plt.figure(figsize=(8, 6))
labels = count.index
sizes = count.values
plt.pie(sizes, labels=labels, autopct='%1.1f%%', startangle=140)
plt.axis('equal')
plt.title('Jumlah Pengguna berdasarkan Kepemilikan Anak')
plt.show()

Terlihat bahwa mayoritas pengguna sudah memiliki anak, Selanjutnya adalah mencari rata-rata banyak anak yang dimiliki oleh pengguna yang memiliki anak.

In [ ]:
# Seleksi data yang memiliki anak
haveChild = cleanedTraining[cleanedTraining['memiliki_anak'] == 'Memiliki']

# Penghitungan rata-rata anak remaja yang dimiliki
teenageChild = haveChild['jumlah_anak_remaja'].mean()

# Penghitungan rata-rata anak balita yang dimiliki
babyChild = haveChild['jumlah_anak_balita'].mean()

# Penghitungan rata-rata anak yang dimiliki
meanChild = (haveChild['jumlah_anak_balita'] + haveChild['jumlah_anak_remaja']).mean()

print('Rata-rata anak remaja yang dimiliki setiap pengguna yang memiliki anak adalah', round(teenageChild), "anak.")
print('Rata-rata anak balita yang dimiliki setiap pengguna yang memiliki anak adalah', round(babyChild), "anak.")
print('Rata-rata anak yang dimiliki setiap pengguna yang memiliki anak adalah', round(meanChild), "anak.")

Kesimpulan dari hasil analisis di atas bahwa rata-rata anak yang dimiliki oleh pengguna yang memiliki anak balita maupun remaja adalah **1**. Namun, untuk keseluruhan rata-rata adalah **2**.

### Pertanyaan 5

Pada sesi ini akan dilakukan analisis pendapatan pengguna berdasarkan segmentasi pendapatan, dengan menganalisis kolom **Pendapatan** dan **Segmentasi Pendapatan**

In [ ]:
# Grouping berdasarkan segmentasi pendapatan
grpBySegment = cleanedTraining.groupby('segmentasi_pendapatan')['pendapatan'].mean()

# Membulatkan hingga format puluhan juta
grpBySegment = round(grpBySegment / 1000000, 1)


Setelah mendapatkan nilai rata-rata pada setiap segmentasi, berikut adalah hasil visualisasi data beserta nilai aktualnya:

In [ ]:
# Plotting bar chart vertikal
plt.figure(figsize=(13, 6))
bars = grpBySegment.plot(kind='bar', rot=0)
plt.xlabel('Segmentasi Pendapatan')
plt.ylabel('Pendapatan (Juta)')
plt.title('Pendapatan Rata-rata Berdasarkan Segmentasi Pendapatan')
plt.legend().remove()  # Menghapus legend

# Menambahkan nilai aktual pada setiap bar
for bar, nilai in zip(bars.patches, grpBySegment):
    plt.text(bar.get_x() + bar.get_width() / 2, bar.get_height(),
             f'{nilai} Juta',
             ha='center', va='bottom')

plt.show()

Kesimpulan dari hasil analisis di atas, dapat disimpulkan bahwa rata-rata pendapatan tertinggi adalah lapisan segmentasi **Sangat Tinggi** dengan nilai **167 Juta**.

### Pertanyaan 6

Pada sesi ini akan dilakukan proses analisis statistik pembelian pengguna berdasarkan pendapatan. Kolom yang dilakukan analisis yaitu Kolom **Segmentasi Pendapatan**, **Pendapatan**, kolom-kolom pembelanjaan, dan **Pembelian Diskon**.

In [ ]:
# Mengambil kolom-kolom yang digunakan
# Data Training
buyingDfTrain = cleanedTraining[['segmentasi_pendapatan','pendapatan', 'belanja_buah', 
                            'belanja_daging', 'belanja_ikan', 'belanja_kue', 
                            'pembelian_diskon']]
# Data Testing
buyingDfTesting = cleanedTesting[['segmentasi_pendapatan','pendapatan', 'belanja_buah', 
                            'belanja_daging', 'belanja_ikan', 'belanja_kue', 
                            'pembelian_diskon']]

In [ ]:
# Mengambil angka kuartil setiap segmen pendapatan
# Grouping berdasarkan segmentasi
# Data Training
grpBySegment = buyingDfTrain.groupby('segmentasi_pendapatan')['pendapatan']
# Hitung angka kuartil
quartiles = grpBySegment.quantile([0.25, 0.5, 1])
# Mengubah menjadi dataframe
quartilesDf = pd.DataFrame(round(quartiles))

# Data Testing
grpBySegmentTest = buyingDfTesting.groupby('segmentasi_pendapatan')['pendapatan']
# Hitung angka kuartil
quartilesTest = grpBySegmentTest.quantile([0.25, 0.5, 1])
# Mengubah menjadi dataframe
quartilesDfTest = pd.DataFrame(round(quartilesTest))

quartilesDf

Setelah mengetahui nilai di setiap kuartil segmentasi, maka selanjutnya membuat label untuk membuat sub segmentasi pada setiap kuartil yang ada. Hal ini bertujuan untuk mengetahui kebiasaan pengguna dalam berbelanja berdasarkan rentang pendapatannya.

In [ ]:
# Membuat sub-segmentasi berdasarkan kuartil per segmentasi pendapatan
# Buat dictionary kuartil
# Data Training
quartilesDict = {
    'Rendah': {'25%': quartilesDf.loc['Rendah', 0.25][0], '50%': quartilesDf.loc['Rendah', 0.50][0], '100%': quartilesDf.loc['Rendah', 1.0][0]},
    'Sedang': {'25%': quartilesDf.loc['Sedang', 0.25][0], '50%': quartilesDf.loc['Sedang', 0.50][0], '100%': quartilesDf.loc['Sedang', 1.0][0]},
    'Tinggi': {'25%': quartilesDf.loc['Tinggi', 0.25][0], '50%': quartilesDf.loc['Tinggi', 0.50][0], '100%': quartilesDf.loc['Tinggi', 1.0][0]},
    'Sangat Tinggi': {'25%': quartilesDf.loc['Sangat Tinggi', 0.25][0], '50%': quartilesDf.loc['Sangat Tinggi', 0.50][0], 
                      '100%': quartilesDf.loc['Sangat Tinggi', 1.0][0]}
}

# Data Testing
quartilesDictTest = {
    'Rendah': {'25%': quartilesDfTest.loc['Rendah', 0.25][0], '50%': quartilesDfTest.loc['Rendah', 0.50][0], '100%': quartilesDfTest.loc['Rendah', 1.0][0]},
    'Sedang': {'25%': quartilesDfTest.loc['Sedang', 0.25][0], '50%': quartilesDfTest.loc['Sedang', 0.50][0], '100%': quartilesDfTest.loc['Sedang', 1.0][0]},
    'Tinggi': {'25%': quartilesDfTest.loc['Tinggi', 0.25][0], '50%': quartilesDfTest.loc['Tinggi', 0.50][0], '100%': quartilesDfTest.loc['Tinggi', 1.0][0]},
    'Sangat Tinggi': {'25%': quartilesDfTest.loc['Sangat Tinggi', 0.25][0], '50%': quartilesDfTest.loc['Sangat Tinggi', 0.50][0], 
                      '100%': quartilesDfTest.loc['Sangat Tinggi', 1.0][0]}
}
# Buat fungsi transformasi sub-segmentasi
def subSegment(row, dictionary):
    if row['pendapatan'] <= dictionary[row['segmentasi_pendapatan']]['25%']:
        return 'Bawah'
    elif row['pendapatan'] > dictionary[row['segmentasi_pendapatan']]['25%'] and row['pendapatan'] <= dictionary[row['segmentasi_pendapatan']]['50%']:
        return 'Menengah'
    elif row['pendapatan'] > dictionary[row['segmentasi_pendapatan']]['50%'] and row['pendapatan'] <= dictionary[row['segmentasi_pendapatan']]['100%']:
        return 'Atas'
    else :
        return 'Menengah'

In [ ]:
# Aplikasikan fungsi ke dataframe
# Data Training
buyingDfTrain['sub_segmentasi'] = buyingDfTrain.apply(lambda row: subSegment(row, quartilesDict), axis=1)
# Data Testing
buyingDfTesting['sub_segmentasi'] = buyingDfTesting.apply(lambda row: subSegment(row, quartilesDictTest), axis=1)

Setelah membuat sub segmentasi, maka selanjutnya adalah melihat hubungan sub segmentasi dengan belanja yang dilakukan.

In [ ]:
# Membuat list kolom belanja
kolomBelanja = ['belanja_buah', 'belanja_daging', 'belanja_ikan', 'belanja_kue']

# Membuat dictionary untuk menyimpan nilai mean dan std
nilai_statistik = {}

# Loop untuk setiap segmentasi pendapatan
for segmen in buyingDfTrain['segmentasi_pendapatan'].unique():
    # Loop untuk setiap sub-segmentasi
    for subsegmen in buyingDfTrain['sub_segmentasi'].unique():
        # Membuat kunci untuk dictionary
        kunci = f"{segmen}-{subsegmen}"
        
        # Membuat kondisi untuk filtering data
        kondisi = (buyingDfTrain['segmentasi_pendapatan'] == segmen) & (buyingDfTrain['sub_segmentasi'] == subsegmen)
        
        # Membuat dictionary untuk menyimpan nilai mean dan std dari setiap kolom belanja
        nilai_statistik[kunci] = {}
        for kolom in kolomBelanja:
            # Menghitung mean dan std
            mean = buyingDfTrain[kondisi][kolom].mean()
            std = buyingDfTrain[kondisi][kolom].std()
            # Menyimpan nilai mean dan std ke dalam dictionary
            nilai_statistik[kunci][f"mean_{kolom}"] = mean
            nilai_statistik[kunci][f"std_{kolom}"] = std


In [ ]:
# Membuat list segmentasi pendapatan dan sub-segmentasi
segmentasi_pendapatan = ['Rendah', 'Sedang', 'Tinggi', 'Sangat Tinggi']
sub_segmentasi = ['Bawah', 'Menengah', 'Atas']

# Buat Fungsi Plot 
def plotStats(data, colBuying, incomeSegment, subSegment, stats):
    # Mengatur lebar bar
    bar_width = 0.2

    # Membuat posisi untuk setiap segmentasi pendapatan dan sub-segmentasi
    index = np.arange(len(incomeSegment))
    index_sub = [index - bar_width, index, index + bar_width]

    # Membuat bar chart
    plt.figure(figsize=(10, 6))

    # Loop untuk setiap sub-segmentasi
    for i, subsegmen in enumerate(subSegment):
        # Membuat bar untuk setiap segmentasi pendapatan
        plt.bar(index_sub[i], [data[f'{pendapatan}-{subsegmen}'][f'{stats}_{colBuying}'] for pendapatan in incomeSegment], 
                bar_width, label=subsegmen)

    # Mengatur label pada sumbu x
    plt.xticks(index, incomeSegment)

    # Mengatur label pada sumbu y
    plt.ylabel(stats)

    if stats == 'mean':
        statsTitle = 'Rata-Rata'
    else:
        statsTitle = 'Standar Deviasi'
    # Menambahkan judul
    plt.title(f'{statsTitle} {colBuying} Berdasarkan Segmentasi Pendapatan dan Sub-Segmentasi')

    # Menambahkan legenda
    plt.legend()

    # Menampilkan plot
    plt.show()

Setelah mendapatkan setiap nilai statistik yang dibutuhkan yaitu standar deviasi dan rata-rata, untuk mengetahui pola datanya perlu dilakukan visualisasi data. Berikut adalah visualisasinya:

1. Kolom Belanja Buah

In [ ]:
# Kolom Belanja Buah (Mean)
jenis_kolom_belanja = 'belanja_buah'
jenis_statistik = 'mean'
# Plot
plotStats(nilai_statistik, jenis_kolom_belanja, segmentasi_pendapatan, sub_segmentasi, jenis_statistik)

In [ ]:
# Kolom Belanja Buah (Std)
jenis_kolom_belanja = 'belanja_buah'
jenis_statistik = 'std'
# Plot
plotStats(nilai_statistik, jenis_kolom_belanja, segmentasi_pendapatan, sub_segmentasi, jenis_statistik)


Dari hasil visualisasi data di atas, dapat ditarik kesimpulan bahwa pembelian paling dominan dilakukan pada segmentasi dan sub-segmentasi:
- Segmentasi **Sangat Tinggi** sub-segmentasi **Bawah**.
- Segmentasi **Tinggi** sub-segmentasi **Atas**
- Segmentasi **Tinggi** sub-segmentasi **Bawah**
- Segmentasi **Sangat Tinggi** sub-segmentasi **Atas**
- Segmentasi **Sangat Tinggi** sub-segmentasi **Menengah**

2. Kolom Belanja Daging

In [ ]:
# Kolom Belanja Daging (Mean)
jenis_kolom_belanja = 'belanja_daging'
jenis_statistik = 'mean'
# Plot
plotStats(nilai_statistik, jenis_kolom_belanja, segmentasi_pendapatan, sub_segmentasi, jenis_statistik)


In [ ]:
# Kolom Belanja Daging (Std)
jenis_kolom_belanja = 'belanja_daging'
jenis_statistik = 'std'
# Plot
plotStats(nilai_statistik, jenis_kolom_belanja, segmentasi_pendapatan, sub_segmentasi, jenis_statistik)


Dari hasil visualisasi data di atas, dapat ditarik kesimpulan bahwa pembelian paling dominan dilakukan pada segmentasi dan sub-segmentasi:
- Segmentasi **Sangat Tinggi** sub-segmentasi **Menengah**.
- Segmentasi **Sangat Tinggi** sub-segmentasi **Atas**
- Segmentasi **Tinggi** sub-segmentasi **Atas**
- Segmentasi **Sangat Tinggi** sub-segmentasi **Bawah**
- Segmentasi **Tinggi** sub-segmentasi **Bawah**

3. Kolom Belanja Ikan

In [ ]:
# Kolom Belanja Ikan (Mean)
jenis_kolom_belanja = 'belanja_ikan'
jenis_statistik = 'mean'
# Plot
plotStats(nilai_statistik, jenis_kolom_belanja, segmentasi_pendapatan, sub_segmentasi, jenis_statistik)


In [ ]:
# Kolom Belanja Ikan (Std)
jenis_kolom_belanja = 'belanja_ikan'
jenis_statistik = 'std'
# Plot
plotStats(nilai_statistik, jenis_kolom_belanja, segmentasi_pendapatan, sub_segmentasi, jenis_statistik)


Dari hasil visualisasi data di atas, dapat ditarik kesimpulan bahwa pembelian paling dominan dilakukan pada segmentasi dan sub-segmentasi:
- Segmentasi **Sangat Tinggi** sub-segmentasi **Bawah**.
- Segmentasi **Sangat Tinggi** sub-segmentasi **Atas**
- Segmentasi **Tinggi** sub-segmentasi **Atas**
- Segmentasi **Tinggi** sub-segmentasi **Bawah**
- Segmentasi **Sangat Tinggi** sub-segmentasi **Menengah**

4. Kolom Belanja Kue

In [ ]:
# Kolom Belanja Kue (Mean)
jenis_kolom_belanja = 'belanja_kue'
jenis_statistik = 'mean'
# Plot
plotStats(nilai_statistik, jenis_kolom_belanja, segmentasi_pendapatan, sub_segmentasi, jenis_statistik)


In [ ]:
# Kolom Belanja Kue (Std)
jenis_kolom_belanja = 'belanja_kue'
jenis_statistik = 'std'
# Plot
plotStats(nilai_statistik, jenis_kolom_belanja, segmentasi_pendapatan, sub_segmentasi, jenis_statistik)


Dari hasil visualisasi data di atas, dapat ditarik kesimpulan bahwa pembelian paling dominan dilakukan pada segmentasi dan sub-segmentasi:
- Segmentasi **Sangat Tinggi** sub-segmentasi **Atas**
- Segmentasi **Sangat Tinggi** sub-segmentasi **Bawah**.
- Segmentasi **Sangat Tinggi** sub-segmentasi **Menengah**
- Segmentasi **Tinggi** sub-segmentasi **Atas**
- Segmentasi **Tinggi** sub-segmentasi **Menengah**

Dari analisis di atas dapat ditarik kesimpulan bahwa:

- Rata-rata pengeluaran pengguna untuk berbelanja relatif terhadap pendapatan mereka, ditandai juga dengan standar deviasi yang tidak terlalu jauh. Maka variasi data yang ada tidak begitu menyebar. 
- Rata-rata pengeluaran terbanyak terdapat pada **sub-segmentasi Atas** disusul **sub-segmentasi Rendah** pada **segmentasi pendapatan Sangat Tinggi**. Dengan range pendapatan **165 - 245 Juta** dan **148 - 155 Juta**.
- Terdapat segmentasi-segmentasi pendapatan yang memiliki potensi untuk berbelanja, dilihat dari standar deviasi yang menjanjikan. Berdasarkan urutan tertinggi yaitu:
  1. Segmentasi pendapatan **Tinggi** dengan sub-segmentasi **Atas**. Pada range pendapatan **131 - 147 Juta**.
  2. Segmentasi pendapatan **Tinggi** dengan sub-segmentasi **Bawah**. Pada range pendapatan **114 - 121 Juta**.
  3. Setiap sub-segmentasi di segmentasi pendapatan **Sedang**.
  
  Pada golongan-golongan ini, perlu untuk dilakukan promosi lebih agar mendorong pengguna untuk melakukan perbelanjaan.

- Pada segmentasi pendapatan **Rendah**, pengeluaran untuk belanja paling rendah. 

Selanjutnya adalah menganalisa pembelian dengan parameter diskon, untuk mengetahui bagaimana kebiasaan pengguna dalam berbelanja. Apakah ketika sedang diskon atau tidak, serta mengetahui segmentasi mana saja yang dominan berbelanja ketika sedang ada diskon.

In [ ]:
# Mengambil kolom yang relevan untuk dataframe baru
buyOnDiscDf = buyingDfTrain[['segmentasi_pendapatan', 'sub_segmentasi', 'pembelian_diskon']]
buyOnDiscDf.head()

In [ ]:
nilaiDiskon = {}
kolomDiskon = 'pembelian_diskon'
# Loop untuk setiap segmentasi pendapatan
for segmen in buyOnDiscDf['segmentasi_pendapatan'].unique():
    # Loop untuk setiap sub-segmentasi
    for subsegmen in buyOnDiscDf['sub_segmentasi'].unique():
        # Membuat kunci untuk dictionary
        kunci = f"{segmen}-{subsegmen}"
        
        # Membuat kondisi untuk filtering data
        kondisi = (buyOnDiscDf['segmentasi_pendapatan'] == segmen) & (buyOnDiscDf['sub_segmentasi'] == subsegmen)
        
        # Membuat dictionary untuk menyimpan nilai mean dan std dari setiap kolom belanja
        nilaiDiskon[kunci] = {}

        # Menghitung mean dan std
        mean = buyOnDiscDf[kondisi][kolomDiskon].mean()
        std = buyOnDiscDf[kondisi][kolomDiskon].std()
        # Menyimpan nilai mean dan std ke dalam dictionary
        nilaiDiskon[kunci][f"mean_{kolomDiskon}"] = mean
        nilaiDiskon[kunci][f"std_{kolomDiskon}"] = std

Setelah mengambil nilai statistik (mean dan std) pada kolom **Pembelian Diskon** berdasarkan segmentasi dan sub-segmentasi pendapatan, selanjutnya melakukan visualisasi data.

In [ ]:
jenis_kolom_belanja = 'pembelian_diskon'
jenis_statistik = 'mean'
# Plot
plotStats(nilaiDiskon, jenis_kolom_belanja, segmentasi_pendapatan, sub_segmentasi, jenis_statistik)

In [ ]:
jenis_kolom_belanja = 'pembelian_diskon'
jenis_statistik = 'std'
# Plot
plotStats(nilaiDiskon, jenis_kolom_belanja, segmentasi_pendapatan, sub_segmentasi, jenis_statistik)

Dari hasil analisa pembelian berdasarkan diskon di atas, dapat disimpulkan bahwa:

- Rata-rata pembelian ketika diskon terbanyak terdapat pada segmentasi pendapatan **Sedang** dengan sub-segmentasi **Bawah** dan **Atas** dengan range pendapatan **82 - 93 Juta** dan **101 - 114 Juta**.

Terdapat beberapa segmentasi dan sub-segmentasi yang berpotensi berbelanja ketika diskon berurutan yaitu:

- Segmentasi pendapatan **Tinggi** dengan sub-segmentasi **Menengah** dan **Bawah**, dengan range pendapatan **122 - 131 Juta** dan **114 - 121 Juta**.
- Segmentasi pendapatan **Rendah** dengan sub-segmentasi **Bawah** dan **Atas**, dengan range pendapatan **5 - 46 Juta** dan **62 - 82 Juta**.

Maka langkah selanjutnya yaitu memberikan promosi diskon dengan frekuensi dari yang tertinggi ke terendah berdasarkan urutan hasil analisis di atas. Yaitu dimulai dari rata-rata tertinggi, dilanjutkan dengan pengguna berpotensi. Serta jalur promosi yang paling diunggulkan, apakah secara luring atau daring.

### Pertanyaan 7

Pada sesi ini akan dilakukan analisis **keluhan pengguna** ketika berbelanja dengan cara **daring (Pembelian Web)** maupun **luring (Pembelian Toko)**. Maka kolom yang digunakan adalah kolom **Segmentasi Pendapatan**, **Sub Segmentasi**, **Pembelian Web**, **Pembelian Toko**, dan **Keluhan**. 

In [ ]:
# Menggabungkan data buying dengan data cleaned training
# Data Training
cleanedTraining['sub_segmentasi'] = buyingDfTrain['sub_segmentasi']
# Data Testing
cleanedTesting['sub_segmentasi'] = buyingDfTesting['sub_segmentasi']

# Mengambil kolom yang diperlukan
worriedBuyDf = cleanedTraining[['segmentasi_pendapatan', 'sub_segmentasi', 'pembelian_web', 'pembelian_toko', 'keluhan']]
worriedBuyDf.head()

In [ ]:
# Mengambil data pengguna yang memiliki keluhan
withWorryDf = worriedBuyDf[worriedBuyDf['keluhan'] == 1]
# Mengambil data pengguna yang tidak memiliki keluhan
withNoWorryDf = worriedBuyDf[worriedBuyDf['keluhan'] == 0]

# Menampilkan banyak pengguna yang memiliki keluhan
print('Banyak pengguna yang memiliki keluhan ketika berbelanja adalah sebanyak', withWorryDf.shape[0], 'pengguna.')
# Menampilkan banyak pengguna yang tidak memiliki keluhan
print('Banyak pengguna yang tidak memiliki keluhan ketika berbelanja adalah sebanyak', withNoWorryDf.shape[0] - withWorryDf.shape[0], 'pengguna.')


In [ ]:
# Rata-rata pembelian dengan cara daring (web)
meanWeb = withWorryDf.pembelian_web.mean()
# Rata-rata pembelian dengan cara luring (toko)
meanMarket = withWorryDf.pembelian_toko.mean()

if meanWeb < meanMarket:
    print('Keluhan ada pada saat pembelian di toko (luring)')
else:
    print('Keluhan ada pada saat pembelian di web (daring)')

Dari hasil analisis di atas, dapat disimpulkan bahwa keluhan berada pada **Toko**. Pengguna yang memiliki keluhan cenderung berbelanja di **Web**. Selanjutnya adalah membandingkan dengan data pengguna yang tidak memiliki keluhan.

In [ ]:
# Rata-rata pembelian dengan cara daring (web)
webNoWorry = withNoWorryDf.pembelian_web.mean()
# Rata-rata pembelian dengan cara luring (toko)
marketNoWorry = withNoWorryDf.pembelian_toko.mean()

print('Pengguna Tanpa Keluhan')
print('Pembelian Web:', round(webNoWorry))
print('Pembelian Toko:', round(marketNoWorry))
print()
print('Pengguna Dengan Keluhan')
print('Pembelian Web:', round(meanWeb))
print('Pembelian Toko:', round(meanMarket))

Dari hasil perbandingan di atas dapat disimpulkan bahwa **pembelian dengan toko terdapat masalah pada pengguna-pengguna tertentu**. Pengguna cenderung berbelanja di **Web** yang dinilai lebih efektif.

Maka selanjutnya dapat menstrategikan untuk menggalakkan promosi baik dari segi diskon maupun barang **di Web** dengan proporsi lebih, daripada di **Toko**.

### Pertanyaan 8

Pada sesi ini akan dilakukan analisis distribusi pendaftaran anggota pada setiap pengguna berdasarkan analisis statistik dan demografis yang sudah dilakukan. Kolom yang digunakan adalah kolom **Tanggal Menjadi Anggota**, **Rentang Usia**, **Segmentasi Pendapatan**, dan **Sub-Segmentasi Pendapatan**

In [ ]:
# Ambil data dengan kolom yang digunakan
memberDf = cleanedTraining[['segmentasi_pendapatan', 'sub_segmentasi', 'rentang_usia', 'tanggal_menjadi_anggota']]
memberDf.head()

In [ ]:
# Ambil data pengguna yang sudah menjadi member
nowMemberDf = memberDf[memberDf['tanggal_menjadi_anggota'] != 'Unknown']
nowMemberDf.head()

In [ ]:
# Menampilkan rentang usia yang menjadi anggota
print('Rentang-rentang usia anggota:', ', '.join(nowMemberDf.rentang_usia.unique()))

# Menampilkan segmentasi dan sub-segmentasi yang ada
print('Dengan daftar segmentasi dan sub-segmentasi pendapatan sebagai berikut:\n')
for segmen in nowMemberDf.segmentasi_pendapatan.unique():
    for sub in nowMemberDf.sub_segmentasi.unique():
        print(f'{segmen}-{sub}')
    print()

Sebab lapisan segmentasi yang ada cukup luas dan mencakup keseluruhan, maka analisis akan dipersempit pada segmentasi pendapatan dan rentang usia saja.

In [ ]:
# Membuat subplot dengan format 1 baris 2 kolom
plt.figure(figsize=(15, 6))

# Plotting pie chart untuk segmentasi pendapatan
plt.subplot(1, 2, 1)
countSegment = nowMemberDf['segmentasi_pendapatan'].value_counts()
labels = countSegment.index
sizes = countSegment.values
plt.pie(sizes, labels=labels, autopct='%1.1f%%', startangle=140)
plt.axis('equal')
plt.title('Rasio Anggota berdasarkan Segmentasi Pendapatan Pengguna')

# Plotting bar chart untuk rentang usia
plt.subplot(1, 2, 2)
countAge = nowMemberDf['rentang_usia'].value_counts()
rot = 0
countAge.plot(kind='bar', title='Jumlah Kolom', rot=rot)

# Mengubah judul sumbu x dan y
plt.xlabel('Rentang Usia')  # Judul sumbu x
plt.ylabel('Jumlah Data')  # Judul sumbu y
plt.title('Jumlah Anggota berdasarkan Rentang Usia')  # Judul plot

# Menampilkan plot
plt.tight_layout()
plt.show()


Dari hasil visualisasi rasio keanggotaan berdasarkan segmentasi pendapatan di atas, dapat disimpulkan bahwa mayoritas pengguna yang sudah menjadi anggota adalah pengguna dengan rentang usia **Dewasa** dan dengan segmentasi pendapatan **Rendah**. Yang notabene rentang usia **Dewasa** sudah berpenghasilan.

Hal ini berhubungan dengan adanya keuntungan lebih ketika menjadi anggota, yaitu seperti mendapatkan diskon dan lain-lain.

### Pertanyaan 9



In [ ]:
# Membuat list kolom belanja
kolomBelanja = ['belanja_buah', 'belanja_daging', 'belanja_ikan', 'belanja_kue']

# Membuat dictionary untuk menyimpan nilai mean dan std
nilai_statistik = {}

# Loop untuk setiap segmentasi pendapatan
for status in cleanedTraining['status_pernikahan'].unique():
    # Loop untuk setiap sub-segmentasi
    for child in cleanedTraining['memiliki_anak'].unique():
        # Membuat kunci untuk dictionary
        kunci = f"{status}-{child}"
        
        # Membuat kondisi untuk filtering data
        kondisi = (cleanedTraining['status_pernikahan'] == status) & (cleanedTraining['memiliki_anak'] == child)
        
        # Membuat dictionary untuk menyimpan nilai mean dan std dari setiap kolom belanja
        nilai_statistik[kunci] = {}
        for kolom in kolomBelanja:
            # Menghitung mean dan std
            mean = buyingDfTrain[kondisi][kolom].mean()
            std = buyingDfTrain[kondisi][kolom].std()
            # Menyimpan nilai mean dan std ke dalam dictionary
            nilai_statistik[kunci][f"mean_{kolom}"] = mean
            nilai_statistik[kunci][f"std_{kolom}"] = std

**1. Belanja Buah**

In [ ]:
# Membuat list status pernikahan dan memiliki anak
status_pernikahan = cleanedTraining['status_pernikahan'].unique().tolist()
memiliki_anak = ['Memiliki', 'Tidak Memiliki']

jenis_kolom_belanja = 'belanja_buah'
jenis_statistik = 'mean'

# Plot
plotStats(nilai_statistik, jenis_kolom_belanja, status_pernikahan, memiliki_anak, jenis_statistik)

In [ ]:
jenis_kolom_belanja = 'belanja_buah'
jenis_statistik = 'std'

# Plot
plotStats(nilai_statistik, jenis_kolom_belanja, status_pernikahan, memiliki_anak, jenis_statistik)

**2. Belanja Daging**

In [ ]:
jenis_kolom_belanja = 'belanja_daging'
jenis_statistik = 'mean'

# Plot
plotStats(nilai_statistik, jenis_kolom_belanja, status_pernikahan, memiliki_anak, jenis_statistik)

In [ ]:
jenis_kolom_belanja = 'belanja_daging'
jenis_statistik = 'std'

# Plot
plotStats(nilai_statistik, jenis_kolom_belanja, status_pernikahan, memiliki_anak, jenis_statistik)

**3. Belanja Ikan**

In [ ]:
jenis_kolom_belanja = 'belanja_ikan'
jenis_statistik = 'mean'

# Plot
plotStats(nilai_statistik, jenis_kolom_belanja, status_pernikahan, memiliki_anak, jenis_statistik)

In [ ]:
jenis_kolom_belanja = 'belanja_ikan'
jenis_statistik = 'std'

# Plot
plotStats(nilai_statistik, jenis_kolom_belanja, status_pernikahan, memiliki_anak, jenis_statistik)

**4. Belanja Kue**

In [ ]:
jenis_kolom_belanja = 'belanja_kue'
jenis_statistik = 'mean'

# Plot
plotStats(nilai_statistik, jenis_kolom_belanja, status_pernikahan, memiliki_anak, jenis_statistik)

In [ ]:
jenis_kolom_belanja = 'belanja_kue'
jenis_statistik = 'std'

# Plot
plotStats(nilai_statistik, jenis_kolom_belanja, status_pernikahan, memiliki_anak, jenis_statistik)

Dari hasil analisis di atas, terdapat potensi penjualan terhadap pembelian pelanggan yang **Tidak Memiliki Anak**. Dengan rincian sebagai berikut:

- Pembelian **Belanja Buah**, dominan kategori status pernikahan secara berurutan yaitu **Menikah** dan **Cerai**.
- Pembelian **Belanja Daging**, dominan kategori status pernikahan secara berurutan yaitu **Cerai Mati** dan **Rencana Menikah**.
- Pembelian **Belanja Ikan**, sedikit berbeda karena perbedaan nilai mean dan std yang cukup signifikan, maka dominan kategori status pernikahan secara berurutan yaitu **Rencana Menikah** dan **Sendiri**. Karena pertimbangan nilai std yang lebih seimbang.
- Pembelian **Belanja Kue**, dominan kategori status pernikahan secara berurutan yaitu **Sendiri** dan **Cerai**.

Selanjutnya dapat dilakukan promosi lebih pada kategori-kategori di atas, pada setiap kategori belanja yang ada.

## Feature Extraction

Sesi ini dilakukan analisis hubungan nilai statistik dan demografis pengguna dengan banyak promosi yang sudah dilakukan sebelumnya. Tujuan dari analisis ini adalah untuk memberikan *insight* agar promosi yang dilakukan selanjutnya dapat dilaksanakan secara maksimal. Pada sesi ini diperlukan proses-proses yang dilakukan yaitu **Ekstraksi Fitur** untuk memaksimalkan fitur-fitur yang digunakan ke dalam model. 


In [ ]:
cleanedTraining.info()

Dari setiap kolom yang ditampilkan, dan hasil dari analisis dari pertanyaan-pertanyaan sebelumnya. Maka pada sesi ini akan menjelaskan menggunakan analisis statistik deskriptif untuk membuat strategi dalam memaksimalkan data, yang nantinya akan dilakukan proses prediksi.

Disini kami menggunakan beberapa variabel untuk melakukan iterasi dalam melakukan analisis, yaitu:

1. Segmentasi Pendapatan
2. Kepemilikan Anak
3. Rentang Usia
4. Keluhan
5. Tanggal Menjadi Anggota

Serta variabel target sebagai berikut:

1. Belanja (Buah, Daging, Ikan, dan Kue)
2. Pembelian Diskon
3. Platform Pembelian (Web dan Toko)

### Pendapatan dengan Belanja

In [ ]:
def stuffPromote(df):
    # Looping tiap baris data
    for index, row in df.iterrows():
        # Kondisi tiap kolom belanja
        if row['segmentasi_pendapatan'] == 'Sangat Tinggi' and row['sub_segmentasi'] == 'Bawah':
            # Belanja Buah
            df.at[index, 'promosi_buah'] = 5
            # Belanja Daging
            df.at[index, 'promosi_daging'] = 2
            # Belanja Ikan
            df.at[index, 'promosi_ikan'] = 5
            # Belanja Kue
            df.at[index, 'promosi_kue'] = 4
        elif row['segmentasi_pendapatan'] == 'Tinggi' and row['sub_segmentasi'] == 'Atas':
            # Belanja Buah
            df.at[index, 'promosi_buah'] = 4
            # Belanja Daging
            df.at[index, 'promosi_daging'] = 3
            # Belanja Ikan
            df.at[index, 'promosi_ikan'] = 3
            # Belanja Kue
            df.at[index, 'promosi_kue'] = 2
        elif row['segmentasi_pendapatan'] == 'Tinggi' and row['sub_segmentasi'] == 'Bawah':
            # Belanja Buah
            df.at[index, 'promosi_buah'] = 3
            # Belanja Daging
            df.at[index, 'promosi_daging'] = 1
            # Belanja Ikan
            df.at[index, 'promosi_ikan'] = 2
            # Belanja Kue
            df.at[index, 'promosi_kue'] = 0
        elif row['segmentasi_pendapatan'] == 'Sangat Tinggi' and row['sub_segmentasi'] == 'Atas':
            # Belanja Buah
            df.at[index, 'promosi_buah'] = 2
            # Belanja Daging
            df.at[index, 'promosi_ikan'] = 4
            # Belanja Ikan
            df.at[index, 'promosi_daging'] = 4
            # Belanja Kue
            df.at[index, 'promosi_kue'] = 5
        elif row['segmentasi_pendapatan'] == 'Sangat Tinggi' and row['sub_segmentasi'] == 'Menengah':
            # Belanja Buah
            df.at[index, 'promosi_buah'] = 1
            # Belanja Daging
            df.at[index, 'promosi_daging'] = 5
            # Belanja Ikan
            df.at[index, 'promosi_ikan'] = 1
            # Belanja Kue
            df.at[index, 'promosi_kue'] = 3
        elif row['segmentasi_pendapatan'] == 'Tinggi' and row['sub_segmentasi'] == 'Menengah':
            # Belanja Buah
            df.at[index, 'promosi_buah'] = 0
            # Belanja Daging
            df.at[index, 'promosi_daging'] = 0
            # Belanja Ikan
            df.at[index, 'promosi_ikan'] = 0
            # Belanja Kue
            df.at[index, 'promosi_kue'] = 1
        else:
            # Belanja Buah
            df.at[index, 'promosi_buah'] = 0
            # Belanja Daging
            df.at[index, 'promosi_daging'] = 0
            # Belanja Ikan
            df.at[index, 'promosi_ikan'] = 0
            # Belanja Kue
            df.at[index, 'promosi_kue'] = 0

In [ ]:
# Data Training
stuffPromote(cleanedTraining)
# Data Testing
stuffPromote(cleanedTesting)

### Keluhan

In [ ]:
def complaints(df):
    # Looping tiap baris data
    for index, row in df.iterrows():
        # Cek keluhan
        if row['keluhan'] == 1:
            if row['pembelian_toko'] > row['pembelian_web']:
                df.at[index, 'prior_platform_promosi'] = 0
            else:
                df.at[index, 'prior_platform_promosi'] = 1
        else:
            df.at[index, 'prior_platform_promosi'] = 1

In [ ]:
# Data Training
complaints(cleanedTraining)
# Data Testing
complaints(cleanedTesting)

### Kepemilikan Anak

In [ ]:
# List kolom promosi
kolomPromosi = ['promosi_buah', 'promosi_daging', 'promosi_ikan', 'promosi_kue']

def marriedAndChild(df):
    # Looping tiap baris data
    for index, row in df.iterrows():
        # Cek apakah memiliki anak
        if row['memiliki_anak'] == 'Tidak Memiliki':
            # Looping isi list kolom promosi
            for col in kolomPromosi:
                if row['status_pernikahan'] == 'Menikah' and col == 'promosi_buah':
                    df.at[index, col] += 1
                elif row['status_pernikahan'] == 'Cerai' and col == 'promosi_buah':
                    df.at[index, col] += 0.5
                elif row['status_pernikahan'] == 'Cerai Mati' and col == 'promosi_daging':
                    df.at[index, col] += 1
                elif row['status_pernikahan'] == 'Rencana Menikah' and col == 'promosi_daging':
                    df.at[index, col] += 0.5
                elif row['status_pernikahan'] == 'Rencana Menikah' and col == 'promosi_ikan':
                    df.at[index, col] += 1
                elif row['status_pernikahan'] == 'Sendiri' and col == 'promosi_ikan':
                    df.at[index, col] += 0.5
                if row['status_pernikahan'] == 'Sendiri' and col == 'promosi_ikan':
                    df.at[index, col] += 1
                elif row['status_pernikahan'] == 'Cerai' and col == 'promosi_ikan':
                    df.at[index, col] += 0.5
                else:
                    df.at[index, col] += 0

In [ ]:
# Data Training
marriedAndChild(cleanedTraining)
# Data Testing
marriedAndChild(cleanedTesting)

# Data Preparation

In [ ]:
nonNumeric = cleanedTraining.select_dtypes(exclude=['number']).columns
print(nonNumeric)

In [ ]:
cleanedTraining['pendidikan'].unique()

In [ ]:
def transformPernikahan(row):
    if row['pendidikan'] == "Sarjana" :
        return 0
    elif row['pendidikan'] == "Magister" :
        return 1
    elif row['pendidikan'] == "SMA" :
        return 2
    elif row['pendidikan'] == "SMP" :
        return 3
    elif row['pendidikan'] == "Doktor" :
        return 4

cleanedTraining['pendidikan'] = cleanedTraining.apply(transformPernikahan, axis=1).astype(int)

In [ ]:
cleanedTraining['status_pernikahan'].unique()

In [ ]:
def transformPernikahan(row):
    if row['status_pernikahan'] == "Rencana Menikah" :
        return 0
    elif row['status_pernikahan'] == "Menikah" :
        return 1
    elif row['status_pernikahan'] == "Sendiri" :
        return 2
    elif row['status_pernikahan'] == "Cerai" :
        return 3
    elif row['status_pernikahan'] == "Cerai Mati" :
        return 4

cleanedTraining['status_pernikahan'] = cleanedTraining.apply(transformPernikahan, axis=1).astype(int)

In [ ]:
def transformKeanggotaan(value):
    if value == 'Unknown':
        return 0
    else:
        return 1

cleanedTraining['tanggal_menjadi_anggota'] = cleanedTraining['tanggal_menjadi_anggota'].apply(transformKeanggotaan).astype(int)


In [ ]:
cleanedTraining.sample(3)

In [ ]:
cleanedTraining['segmentasi_pendapatan'].unique()

In [ ]:
def transformSegementasi(row):
    if row['segmentasi_pendapatan'] == "Rendah" :
        return 0
    elif row['segmentasi_pendapatan'] == "Sedang" :
        return 1
    elif row['segmentasi_pendapatan'] == "Tinggi" :
        return 2
    elif row['segmentasi_pendapatan'] == "Sangat Tinggi" :
        return 3

cleanedTraining['segmentasi_pendapatan'] = cleanedTraining.apply(transformSegementasi, axis=1).astype(int)

In [ ]:
cleanedTraining['rentang_usia'].unique()

In [ ]:
def transformUsia(row):
    if row['rentang_usia'] == "Remaja" :
        return 0
    elif row['rentang_usia'] == "Dewasa" :
        return 1
    elif row['rentang_usia'] == "Lanjut Usia" :
        return 2

cleanedTraining['rentang_usia'] = cleanedTraining.apply(transformUsia, axis=1).astype(int)

In [ ]:
cleanedTraining['memiliki_anak'].unique()

In [ ]:
def transformAnak(value):
    if value == 'Tidak Memiliki':
        return 0
    else:
        return 1

cleanedTraining['memiliki_anak'] = cleanedTraining['memiliki_anak'].apply(transformAnak).astype(int)


In [ ]:
cleanedTraining['sub_segmentasi'].unique()

In [ ]:
def transformSubsegmen(row):
    if row['sub_segmentasi'] == "Bawah" :
        return 0
    elif row['sub_segmentasi'] == "Menengah" :
        return 1
    elif row['sub_segmentasi'] == "Atas" :
        return 2

cleanedTraining['sub_segmentasi'] = cleanedTraining.apply(transformSubsegmen, axis=1).astype(int)

# Skenario tambahan untuk Data Test

### Cek Null

In [ ]:
cleanedTesting.head()

In [ ]:
cleanedTesting.shape

In [ ]:
cleanedTesting.info()

In [ ]:
cleanedTesting.describe()

In [ ]:
cleanedTesting.isnull().sum()

In [ ]:
nonNumeric = cleanedTesting.select_dtypes(exclude=['number']).columns
print(nonNumeric)

In [ ]:
cleanedTesting['pendidikan'] = cleanedTesting.apply(transformPernikahan, axis=1).astype(int)
cleanedTesting['status_pernikahan'] = cleanedTesting.apply(transformPernikahan, axis=1).astype(int)
cleanedTesting['tanggal_menjadi_anggota'] = cleanedTesting['tanggal_menjadi_anggota'].apply(transformKeanggotaan).astype(int)
cleanedTesting['segmentasi_pendapatan'] = cleanedTesting.apply(transformSegementasi, axis=1).astype(int)
cleanedTesting['rentang_usia'] = cleanedTesting.apply(transformUsia, axis=1).astype(int)
cleanedTesting['memiliki_anak'] = cleanedTesting['memiliki_anak'].apply(transformAnak).astype(int)
cleanedTesting['sub_segmentasi'] = cleanedTesting.apply(transformSubsegmen, axis=1).astype(int)

In [ ]:
cleanedTesting['is_train'] = False
cleanedTraining['is_train'] = True

In [ ]:
penanda1 = cleanedTraining['is_train']
penanda2 = cleanedTesting['is_train']
penanda3 = cleanedTesting['ID']
cleanedTraining.drop(columns=['is_train'], inplace=True)
cleanedTesting.drop(columns=['is_train'], inplace=True)
cleanedTesting.drop(columns=['ID'], inplace=True)

In [ ]:
cleanedTesting['jumlah_promosi'] = pd.Series(dtype=int)

In [ ]:
cleanedTraining['jumlah_promosi'].unique()

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_recall_fscore_support

In [ ]:
X_train = cleanedTraining.drop(columns=['jumlah_promosi'])
y_train = cleanedTraining['jumlah_promosi']
X_test = cleanedTesting.drop(columns=['jumlah_promosi'])

# Encode categorical data
label_encoder = LabelEncoder()
X_train_encoded = X_train.copy()
X_test_encoded = X_test.copy()
categorical_cols = ['pendidikan', 'status_pernikahan', 'keluhan', 'tanggal_menjadi_anggota', 'segmentasi_pendapatan', 'rentang_usia', 'memiliki_anak', 'sub_segmentasi', 'promosi_buah', 'promosi_daging', 'promosi_ikan', 'promosi_kue', 'prior_platform_promosi']
for col in categorical_cols:
    X_train_encoded[col] = label_encoder.fit_transform(X_train[col])
    X_test_encoded[col] = label_encoder.transform(X_test[col])

# normalisasi datanya
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_encoded)
X_test_scaled = scaler.transform(X_test_encoded)

#nilai asli
y_true = y_train.copy()

# Train model
model = RandomForestClassifier()
model.fit(X_train_scaled, y_train)

# Prediksi ke data testing
y_pred = model.predict(X_test_scaled)

In [ ]:
y_pred

In [ ]:
from sklearn.model_selection import cross_val_score

# Hitung skor validasi silang (cross-validation score)
scores = cross_val_score(model, X_train_scaled, y_train, cv=5, scoring='accuracy')  # Ganti 'accuracy' dengan metrik evaluasi yang sesuai

# Tampilkan skor validasi silang
print(f'Cross-Validation Scores: {scores}')
print(f'Average Cross-Validation Score: {scores.mean()}')

In [ ]:
hasil_prediksi = pd.DataFrame({'ID': penanda3, 'jumlah_promosi': y_pred})

# Tampilkan hasil prediksi
print(hasil_prediksi)

In [ ]:
# Simpan DataFrame hasil_prediksi ke dalam file CSV
hasil_prediksi.to_csv('test_labels.csv', index=False)


In [ ]:
# gabungan = pd.concat([cleanedTesting, cleanedTraining], ignore_index=True)

In [ ]:
# gabungan.sample()

In [ ]:
# penanda = gabungan['is_train']
# gabungan.drop(columns=['is_train'], inplace=True)

In [ ]:
# gabungan.isnull().sum()

In [ ]:
# StandardScaler = StandardScaler()

# dataNormalisasi = StandardScaler.fit_transform(gabungan)
# dataNormalisasi

In [ ]:
# z_scores = np.abs(dataNormalisasi - np.mean(dataNormalisasi, axis=0)) / np.std(dataNormalisasi, axis=0)
# threshold = 23  

# # Dapatkan indeks baris yang mengandung outlier
# outlier_indices = np.where(z_scores > threshold)

# # Hapus baris yang mengandung outlier dari dataNormalisasi
# gabungan = np.delete(dataNormalisasi, outlier_indices[0], axis=0)
# print(gabungan)